### Input: Position Model: MLP

In [87]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

In [88]:
data=pd.read_csv('tictactoe.csv',header=1)
data.head()

,Position,String,Value,Remoteness
0,0,---------,Tie,9
1,2,x--------,Tie,8
2,5,xo-------,Win,5
3,6,-x-------,Tie,8
4,7,ox-------,Tie,7


In [89]:
# 0,1,2: win,lose,tie; 0:_; 1:o; 2:x
# replace the data for training
data=data.replace('Win',0)
data=data.replace('Lose',1)
data=data.replace('Tie',2)
char_mapping = {'x': 2, 'o': 1, '-': 0}
data['String'] = data['String'].apply(lambda s: [char_mapping[c] for c in s])

data.head()


,Position,String,Value,Remoteness
0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",2,9
1,2,"[2, 0, 0, 0, 0, 0, 0, 0, 0]",2,8
2,5,"[2, 1, 0, 0, 0, 0, 0, 0, 0]",0,5
3,6,"[0, 2, 0, 0, 0, 0, 0, 0, 0]",2,8
4,7,"[1, 2, 0, 0, 0, 0, 0, 0, 0]",2,7


In [90]:
hash_table={}
for idx,i in enumerate(data['String']):
    hash_table[tuple(i)]=data['Position'][idx]
hash_table[((0, 0, 0, 0, 0, 0, 0, 0, 0))]

0

In [91]:
a=(data.iloc[0]['Value'],data.iloc[0]['Remoteness'])
a=torch.tensor(a)

In [92]:
from torch.utils.data import Dataset,DataLoader,random_split

class tttDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        features = sample['String']  # 根据你的数据结构来处理特征和标签
        value_label = sample['Value']
        remoteness_label = sample['Remoteness']
        return torch.tensor(features,dtype=torch.float32), torch.tensor(value_label,dtype=torch.long), torch.tensor(remoteness_label,dtype=torch.long)
        # return torch.tensor(features,dtype=torch.float32) ,torch.tensor(label,dtype=torch.long)

In [93]:
dataset = tttDataset(data)
# train_num=int(len(dataset)*0.8)
# test_num=len(dataset)-train_num
# train_dataset,test_dataset=random_split(dataset,[train_num,test_num])
batch_size = 8
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
dataset_loader=DataLoader(dataset,batch_size=batch_size,shuffle=True)

In [94]:
import torch.nn as nn
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.flatten=nn.Flatten()
        self.linear_relu_val = nn.Sequential(
            nn.Linear(9,32),
            nn.ReLU(),
            nn.Linear(32,64),
            nn.ReLU(),
            # nn.Linear(128,32),
            # nn.ReLU(),
            nn.Linear(64,3),
            )
        # ?
        self.linear_relu_rem = nn.Sequential(
            nn.Linear(9,32),
            nn.ReLU(),
            nn.Linear(32,64),
            nn.ReLU(),
            # nn.Linear(64,128),
            # nn.ReLU(),
            # nn.Linear(128,32),
            # nn.ReLU(),
            nn.Linear(64,10),
            )
    def forward(self,inputs):
        # print(inputs.shape)
        # inputs=inputs.unsqueeze(1)
        val_logits=self.linear_relu_val(inputs)
        val_logits=nn.functional.softmax(val_logits,dim=1)
        rem_logits=self.linear_relu_rem(inputs)
        rem_logits=nn.functional.softmax(rem_logits,dim=1)
        return val_logits,rem_logits

In [95]:
learning_rate = 1e-3
# random_seed = 1
# torch.manual_seed(random_seed)
n_epochs = 300
models = MLP().to('cpu')

In [96]:
models.requires_grad_(True)
optimizer=torch.optim.Adam(models.parameters(), lr=learning_rate)
loss_func = nn.CrossEntropyLoss()

In [97]:
models.requires_grad_(True)

for epoch in range(n_epochs):
    models.train(True)
    train_loss=0
    val_acc=0
    rem_acc=0
    for data, val_target,rem_target in dataset_loader:
        optimizer.zero_grad()
        val_outputs,rem_outputs = models(data)
        val_predicted = torch.argmax(val_outputs, dim=1)
        rem_predicted = torch.argmax(rem_outputs, dim=1)
        v_correct = (val_predicted == val_target).sum()
        r_correct = (rem_predicted == rem_target).sum()
        # print(rem_outputs.size(),rem_target.size(),rem_outputs)
        rem_outputs=rem_outputs.squeeze(1)
        loss_val = loss_func(val_outputs, val_target)
        loss_rem = loss_func(rem_outputs, rem_target)
        # print(val_predicted,val_target,rem_outputs,rem_target)
        loss=loss_val+loss_rem
        loss.backward()
        optimizer.step()
       
        train_loss+=loss
        val_acc+=v_correct
        rem_acc+=r_correct
    train_loss = train_loss/(len(train_loader))
    val_acc=val_acc/(len(train_loader)*batch_size)
    rem_acc=rem_acc/(len(train_loader)*batch_size)
    # print('Epoch [{}], Accuracy:{:.4f}, Loss: {:.4f}'
    #             .format(epoch + 1, acc,train_loss.item()))
    print('Epoch [{}],  Loss: {:.4f}, value accuracy:{:.4f},remoteness accuracy:{:.4f}'
                .format(epoch + 1, train_loss.item(),val_acc,rem_acc))

Epoch [1],  Loss: 3.8149, value accuracy:0.6496,remoteness accuracy:0.5527
Epoch [2],  Loss: 3.7603, value accuracy:0.6809,remoteness accuracy:0.5570
Epoch [3],  Loss: 3.7407, value accuracy:0.6957,remoteness accuracy:0.5641
Epoch [4],  Loss: 3.7220, value accuracy:0.7071,remoteness accuracy:0.5762
Epoch [5],  Loss: 3.7123, value accuracy:0.7158,remoteness accuracy:0.5778
Epoch [6],  Loss: 3.7060, value accuracy:0.7235,remoteness accuracy:0.5807
Epoch [7],  Loss: 3.6921, value accuracy:0.7379,remoteness accuracy:0.5828
Epoch [8],  Loss: 3.6833, value accuracy:0.7479,remoteness accuracy:0.5823
Epoch [9],  Loss: 3.6740, value accuracy:0.7541,remoteness accuracy:0.5828
Epoch [10],  Loss: 3.6654, value accuracy:0.7591,remoteness accuracy:0.5867
Epoch [11],  Loss: 3.6570, value accuracy:0.7692,remoteness accuracy:0.5887
Epoch [12],  Loss: 3.6516, value accuracy:0.7733,remoteness accuracy:0.5874
Epoch [13],  Loss: 3.6438, value accuracy:0.7851,remoteness accuracy:0.5899
Epoch [14],  Loss: 3.

In [98]:
torch.save(models.state_dict(), './mlp.pt')
models.load_state_dict(torch.load('./mlp.pt'))
models.eval()

MLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_val): Sequential(
    (0): Linear(in_features=9, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=3, bias=True)
  )
  (linear_relu_rem): Sequential(
    (0): Linear(in_features=9, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [99]:
exception_table=[]
for data, val_target,rem_target in dataset_loader:
    val_outputs,rem_outputs = models(data)
    val_predicted = torch.argmax(val_outputs, dim=1)
    rem_predicted = torch.argmax(rem_outputs, dim=1)
    # print()
    ar_data=np.array(data,dtype=int)
    hash_positions=[hash_table[tuple(i)] for i in ar_data]
    
    delta_val=val_predicted-val_target
    delta_rem=rem_predicted-rem_target
    for idx,i in enumerate(delta_val):
        if i!=0 or delta_rem[idx]!=0:
            exception_table.append([hash_positions[idx],ar_data[idx],int(i),int(delta_rem[idx])])
    # exception_table.append([val_outputs,rem_outputs,val_target,rem_target])

In [100]:
len(exception_table)

2819

In [101]:
csv_file = "exception_delta.csv"

import csv
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    for row in exception_table:
        writer.writerow(row)